# Capstone Project Notebook

#### Import required packages:

In [1]:
import pandas as pd
import numpy as np

In [2]:
# !pip install bs4
# !pip install requests
from bs4 import BeautifulSoup
import requests

### PartA: Obtaining information from wikipedia

In [3]:
#Mark the url and obtain lxml
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = requests.get(url).text
soup = BeautifulSoup(req,'lxml')

In [5]:
#cut out the required table
table = soup.find('table',{'class':'wikitable sortable'})
ths = table.find_all('th')

In [101]:
#Clean the table data and put into a dataframe
rows = table.findAll("tr")
row_lengths = [len(r.findAll(['th', 'td'])) for r in rows]
ncols = max(row_lengths)
nrows = len(rows)
data = []
for i in range(nrows):
    rowD = []
    for j in range(ncols):
        rowD.append('')
    data.append(rowD)

for i in range(len(rows)):
    row = rows[i]
    rowD = []
    cells = row.findAll(["td", "th"])
    for j in range(len(cells)):
        cell = cells[j]

        #lots of cells span cols and rows so lets deal with that
        cspan = int(cell.get('colspan', 1))
        rspan = int(cell.get('rowspan', 1))
        l = 0
        for k in range(rspan):
            # Shifts to the first empty cell of this row
            while data[i + k][j + l]:
                l += 1
            for m in range(cspan):
                cell_n = j + l + m
                row_n = i + k
                # in some cases the colspan can overflow the table, in those cases just get the last item
                cell_n = min(cell_n, len(data[row_n])-1)
                data[row_n][cell_n] += cell.text

    data.append(rowD)

In [118]:
df = pd.DataFrame(data, columns=["Postcode", "Borough", "Neighbourhood"])
#Remove no post code records
df = df[df.Postcode != 'Postcode']
#drop null values
df = df.dropna()
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']
#Clean the Neighbourhood column
df = df.replace('\n', '', regex=True)
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df.loc[df.Neighbourhood == "Not assigned", 'Neighbourhood'] = df['Borough']

In [137]:
#Remove duplicate by merging
df_dup = df[df.duplicated(subset=["Postcode", "Borough"])].groupby(["Postcode", "Borough"])['Neighbourhood'].apply(','.join).reset_index()
df_nodup = df.drop_duplicates()
df_final = pd.concat([df_dup, df_nodup], ignore_index=True, sort = False)

In [138]:
#Checking if there are missing Postcode
print(df_final.Postcode.nunique(),df.Postcode.nunique())

103 103


In [140]:
df_final.shape

(266, 3)